In [1]:
import os
import sys
sys.path.append("../source/")
sys.path.append("../")

from dotenv import load_dotenv

load_dotenv("../.env")

import pinecone
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain

REPLICATE_API_TOKEN = os.environ["REPLICATE_API_TOKEN"]
PINECONE_API_TOKEN = os.environ["PINECONE_API_TOKEN"]

pinecone.init(api_key = PINECONE_API_TOKEN, environment = "gcp-starter")

/home/zero/.cache/pypoetry/virtualenvs/src-TO89gsno-py3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
Loader = PyPDFLoader("../data/5 September 2023 _ Minutes of the Monetary Policy Meeting of the Board _ RBA.pdf")
Documents = Loader.load()

In [3]:
TextSplitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
Texts = TextSplitter.split_documents(Documents)

In [4]:
Embeddings = HuggingFaceEmbeddings()

In [5]:
IndexName = "centralbanksllm"
Index = pinecone.Index(IndexName)
VectorDB = Pinecone.from_documents(Texts, Embeddings, index_name = IndexName)

In [6]:
LLM = Replicate(model = "a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5", input={"temperature": 0.75, "max_length": 3000})

Init param `input` is deprecated, please use `model_kwargs` instead.


In [7]:
QA_Chain = ConversationalRetrievalChain.from_llm(LLM, VectorDB.as_retriever(search_kwargs = {'k': 2}), return_source_documents = True)

In [8]:
ChatHistory = []

while True:
    Query = input('Prompt: ')
    
    if Query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
        
    Result = QA_Chain({'question': Query, 'chat_history': ChatHistory})
    print('Answer: ' + Result['answer'] + '\n')
    ChatHistory.append((Query, Result['answer']))

Prompt:  What were the assessment for the RBA monetary policy meeting of september?


Answer: Based



Prompt:  What were the assessment for the RBA monetary policy meeting of september?


Answer: Sure



Prompt:  What were the main points of the RBA monetary policy meeting of september?


...

Prompt:  What were the main points of the RBA monetary policy meeting of september?


Answer: Yes



Prompt:  who is the president of the united states?


Answer: The answer to the question is that the president of the United States is not mentioned in the given context. The context only mentions the Reserve Bank of Australia and its board minutes. Therefore, the president of the United States is not relevant to the given context.



KeyboardInterrupt: Interrupted by user